In [1]:
! pip install faker pandas numpy


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
! pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0
    Uninstalling pip-25.0:
      Successfully uninstalled pip-25.0


In [9]:
! pip install git+https://github.com/jackruss/faker-medical.git


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/jackruss/faker-medical.git to /private/var/folders/sx/7hgzmj7n3012sjybqk4k0dkm0000gn/T/pip-req-build-ch9mx8yp
  Running command git clone --filter=blob:none --quiet https://github.com/jackruss/faker-medical.git /private/var/folders/sx/7hgzmj7n3012sjybqk4k0dkm0000gn/T/pip-req-build-ch9mx8yp
  Resolved https://github.com/jackruss/faker-medical.git to commit 8f9db5203efa9ce194f3ad7e6abb89157ae6b971
ERROR: git+https://github.com/jackruss/faker-medical.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [10]:
import pandas as pd
import numpy as np
from faker import Faker
from faker.providers.medical import Provider as MedicalProvider
from faker.providers import ssn, person, address, phone_number, medical, date_time
from datetime import datetime, timedelta
import random


ModuleNotFoundError: No module named 'faker_medical'

In [17]:
import pandas as pd
import numpy as np
from faker import Faker
from faker.providers import ssn, person, address, phone_number, date_time
from datetime import datetime, timedelta
import random
import zipfile

# Initialize Faker with providers
fake = Faker()
fake.add_provider(ssn)
fake.add_provider(person)
fake.add_provider(address)
fake.add_provider(phone_number)
fake.add_provider(date_time)

# Configuration
NUM_PATIENTS = 10000
NUM_DOCTORS = 200
NUM_ALLERGIES = 150
NUM_MEDS = 300
START_DATE = datetime(2010, 1, 1)
END_DATE = datetime(2025, 3, 23)

# Initialize data storage
tables = {}

# ========== Lookup Tables ==========
departments = [
    'Cardiology', 'Oncology', 'Pediatrics', 'Neurology', 'Orthopedics',
    'Emergency', 'Surgery', 'Radiology', 'ICU', 'Labor & Delivery',
    'Dermatology', 'Gastroenterology', 'Endocrinology', 'Pulmonology',
    'Nephrology', 'Psychiatry', 'Urology', 'Ophthalmology', 'ENT', 'Hematology'
]
tables['Departments'] = pd.DataFrame({
    'DepartmentID': range(1, 21),
    'DepartmentName': departments,
    'Location': [fake.street_address() for _ in range(20)]
})

specialty_map = {
    'Cardiology': 'Cardiac Care',
    'Oncology': 'Cancer Treatment',
    'Pediatrics': 'Child Health',
    'Neurology': 'Nervous System',
    'Orthopedics': 'Musculoskeletal',
    'Emergency': 'Trauma Care',
    'Surgery': 'Surgical Services',
    'Radiology': 'Medical Imaging',
    'ICU': 'Critical Care',
    'Labor & Delivery': 'Obstetrics',
    'Dermatology': 'Skin Disorders',
    'Gastroenterology': 'Digestive Health',
    'Endocrinology': 'Hormonal Disorders',
    'Pulmonology': 'Respiratory System',
    'Nephrology': 'Kidney Diseases',
    'Psychiatry': 'Mental Health',
    'Urology': 'Urinary System',
    'Ophthalmology': 'Eye Care',
    'ENT': 'Ear/Nose/Throat',
    'Hematology': 'Blood Disorders'
}
tables['DepartmentSpecialty'] = pd.DataFrame({
    'DepartmentID': tables['Departments'].DepartmentID,
    'Specialty': [specialty_map[name] for name in tables['Departments'].DepartmentName]
})

icd10_codes = [
    ('A00-B99', 'Certain infectious and parasitic diseases'),
    ('C00-D49', 'Neoplasms'),
    ('D50-D89', 'Diseases of the blood'),
    ('E00-E89', 'Endocrine, nutritional and metabolic diseases'),
    ('F01-F99', 'Mental, Behavioral and Neurodevelopmental disorders'),
    ('G00-G99', 'Diseases of the nervous system'),
    ('H00-H59', 'Diseases of the eye and adnexa'),
    ('H60-H95', 'Diseases of the ear and mastoid process'),
    ('I00-I99', 'Diseases of the circulatory system'),
    ('J00-J99', 'Diseases of the respiratory system')
]
tables['ICD10Codes'] = pd.DataFrame(icd10_codes, columns=['ICD10Code', 'DiagnosisCategory'])

# ========== Core Entities ==========
tables['Doctors'] = pd.DataFrame([{
    'DoctorID': i+1,
    'FirstName': fake.first_name(),
    'LastName': fake.last_name(),
    'License': f'MD{str(i+1).zfill(6)}',
    'HireDate': fake.date_between(start_date='-20y', end_date='-1y'),
    'DepartmentID': random.choice(tables['Departments'].DepartmentID.tolist())
} for i in range(NUM_DOCTORS)])

patient_data = []
ssn_set = set()
for i in range(NUM_PATIENTS):
    while True:
        ssn = fake.unique.ssn()
        if ssn not in ssn_set:
            ssn_set.add(ssn)
            break
    patient_data.append({
        'PatientID': i+1,
        'FirstName': fake.first_name(),
        'LastName': fake.last_name(),
        'DateOfBirth': fake.date_of_birth(minimum_age=1, maximum_age=100),
        'Gender': random.choice(['Male', 'Female', 'Other']),
        'SSN': ssn
    })
tables['Patients'] = pd.DataFrame(patient_data)

tables['PatientContactInfo'] = pd.DataFrame([{
    'ContactID': i+1,
    'PatientID': i+1,
    'Address': fake.street_address(),
    'City': fake.city(),
    'State': fake.state_abbr(),
    'ZipCode': fake.zipcode(),
    'Phone': fake.phone_number(),
    'Email': fake.email()
} for i in range(NUM_PATIENTS)])

allergy_names = set()
while len(allergy_names) < NUM_ALLERGIES:
    allergy_names.add(f"{fake.word().capitalize()} Allergy")
tables['Allergies'] = pd.DataFrame([{
    'AllergyID': i+1,
    'AllergyName': name,
    'Severity': random.choice(['Low', 'Medium', 'High'])
} for i, name in enumerate(allergy_names)])

med_names = set()
while len(med_names) < NUM_MEDS:
    med_names.add(f"{fake.word().capitalize()}{random.choice(['cin', 'mycin', 'zol', 'pam', 'x'])}")
tables['Medications'] = pd.DataFrame([{
    'MedicationID': i+1,
    'MedicationName': name,
    'Manufacturer': fake.company(),
    'Description': fake.text(max_nb_chars=200)
} for i, name in enumerate(med_names)])

# ========== Relational Data ==========
patient_allergies = []
for patient in tables['Patients'].itertuples():
    num_allergies = np.random.choice([0, 1, 2, 3], p=[0.3, 0.4, 0.2, 0.1])
    allergies = random.sample(
        tables['Allergies'].AllergyID.tolist(),
        k=num_allergies
    )
    for allergy in allergies:
        birth_datetime = datetime.combine(patient.DateOfBirth, datetime.min.time())
        min_start_date = birth_datetime + timedelta(days=365*5)
        start_date = max(min_start_date, START_DATE)
        if start_date < END_DATE:
            patient_allergies.append({
                'PatientID': patient.PatientID,
                'AllergyID': allergy,
                'DateDiagnosed': fake.date_between(
                    start_date=start_date,
                    end_date=END_DATE
                )
            })
tables['PatientAllergies'] = pd.DataFrame(patient_allergies)

diagnoses = []
for code in tables['ICD10Codes'].itertuples():
    for _ in range(50):
        diagnoses.append({
            'DiagnosisID': len(diagnoses)+1,
            'ICD10Code': code.ICD10Code,
            'DiagnosisName': f"{fake.word().capitalize()} {random.choice(['Syndrome', 'Disorder', 'Disease'])}"
        })
tables['Diagnoses'] = pd.DataFrame(diagnoses)

appointments = []
app_id = 1
for patient in tables['Patients'].itertuples():
    num_appointments = np.random.poisson(4) + 1
    for _ in range(num_appointments):
        doctor = random.choice(tables['Doctors'].itertuples())
        birth_datetime = datetime.combine(patient.DateOfBirth, datetime.min.time())
        appointments.append({
            'AppointmentID': app_id,
            'PatientID': patient.PatientID,
            'DoctorID': doctor.DoctorID,
            'AppointmentDateTime': fake.date_time_between(
                start_date=max(
                    doctor.HireDate + timedelta(days=30),
                    birth_datetime + timedelta(days=365*5)
                ),
                end_date=END_DATE
            ),
            'Status': random.choices(
                ['Scheduled', 'Completed', 'Cancelled'],
                weights=[0.1, 0.8, 0.1]
            )[0],
            'Notes': fake.text(max_nb_chars=200) if random.random() > 0.3 else None
        })
        app_id += 1
tables['Appointments'] = pd.DataFrame(appointments)

symptom_list = ['Fever', 'Cough', 'Pain', 'Fatigue', 'Nausea', 'Rash', 'Headache']
treatment_list = ['Rest', 'Antibiotics', 'Surgery', 'Therapy', 'Medication']
tables['MedicalRecords'] = pd.DataFrame([{
    'RecordID': i+1,
    'AppointmentID': appt.AppointmentID,
    'Symptoms': ', '.join(random.sample(symptom_list, random.randint(1, 3))),
    'DiagnosisID': random.choice(tables['Diagnoses'].DiagnosisID.tolist()),
    'Treatment': random.choice(treatment_list),
    'LabResults': fake.text(max_nb_chars=200) if random.random() > 0.5 else None
} for i, appt in enumerate(tables['Appointments'].itertuples())])

prescriptions = []
for appt in tables['Appointments'].itertuples():
    if appt.Status == 'Completed' and random.random() > 0.4:
        num_scripts = random.randint(0, 3)
        for _ in range(num_scripts):
            prescriptions.append({
                'PrescriptionID': len(prescriptions)+1,
                'MedicationID': random.choice(tables['Medications'].MedicationID.tolist()),
                'Dosage': f'{random.randint(1, 500)}mg',
                'Frequency': random.choice(['QD', 'BID', 'TID', 'QID']),
                'StartDate': appt.AppointmentDateTime.date(),
                'EndDate': (appt.AppointmentDateTime + timedelta(days=random.randint(7, 30))).date()
            })
tables['Prescriptions'] = pd.DataFrame(prescriptions)

tables['Insurance'] = pd.DataFrame([{
    'InsuranceID': i+1,
    'PatientID': i+1,
    'CompanyName': fake.company(),
    'PolicyNumber': f'POL{str(i+1).zfill(6)}',
    'CoverageDetails': fake.text(max_nb_chars=300),
    'EffectiveDate': fake.date_between(start_date='-10y', end_date='-1y'),
    'ExpiryDate': fake.date_between(start_date='today', end_date='+5y')
} for i in range(NUM_PATIENTS)])

# ========== Validation ==========
assert tables['Patients'].SSN.nunique() == NUM_PATIENTS
assert tables['Doctors'].License.nunique() == NUM_DOCTORS
assert tables['Insurance'].PolicyNumber.nunique() == NUM_PATIENTS

for table in ['Doctors', 'DepartmentSpecialty']:
    assert tables[table]['DepartmentID'].isin(tables['Departments'].DepartmentID).all()

# ========== Export ==========
with zipfile.ZipFile('medical_data.zip', 'w') as zipf:
    for table_name, df in tables.items():
        filename = f'{table_name}.csv'
        df.to_csv(filename, index=False)
        zipf.write(filename)
        print(f'Generated {len(df)} records in {filename}')

print('Data generation complete. All files zipped to medical_data.zip')

TypeError: can't compare datetime.datetime to datetime.date

In [15]:
! pip install faker-medical

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement faker-medical (from versions: none)
ERROR: No matching distribution found for faker-medical
